In [1]:
import cv2
from ultralytics import YOLO
import os

def run_video_inference(model_path: str, input_video_path: str, output_directory: str) -> None:
    model = YOLO(model_path)
    print("Model loaded.")

    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Error opening video file.")
        return

    # Extract the base name of the input file and create an output file name
    base_name = os.path.basename(input_video_path)
    file_name, _ = os.path.splitext(base_name)
    output_video_path = os.path.join(output_directory, f"{file_name}_processed.mp4")
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))
    print(f"Video writer initialized at {output_video_path}")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("No frames read, breaking loop.")
            break

        results = model(frame)
        print("Inference done.")

        for result in results:
            boxes = result.boxes
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = int(box.cls[0])
                label = model.names[cls]

                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        out.write(frame)
        print("Frame written to file.")

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print("Video processing completed.")

In [2]:
path_to_model = "best.pt"
input_video_path1 = "samples/rumination.mp4"
input_video_path2 = "samples/rumination2.mp4"

output_dir = "outputs"

In [3]:
run_video_inference(path_to_model, input_video_path1, output_dir)

Model loaded.
Video writer initialized at outputs/rumination_processed.mp4
0: 640x384 (no detections), 124.1ms
Speed: 3.0ms preprocess, 124.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)
Inference done.
Frame written to file.
0: 640x384 (no detections), 133.9ms
Speed: 1.9ms preprocess, 133.9ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
Inference done.
Frame written to file.
0: 640x384 (no detections), 132.6ms
Speed: 2.1ms preprocess, 132.6ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
Inference done.
Frame written to file.
0: 640x384 (no detections), 172.2ms
Speed: 2.1ms preprocess, 172.2ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)
Inference done.
Frame written to file.
0: 640x384 (no detections), 182.9ms
Speed: 2.6ms preprocess, 182.9ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)
Inference done.
Frame written to file.
0: 640x384 (no detections), 124.8ms
Speed: 2.5ms preprocess,

In [4]:
run_video_inference(path_to_model, input_video_path2, output_dir)

Model loaded.
Video writer initialized at outputs/rumination2_processed.mp4
0: 640x384 (no detections), 117.0ms
Speed: 1.9ms preprocess, 117.0ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
Inference done.
Frame written to file.
0: 640x384 (no detections), 112.1ms
Speed: 1.9ms preprocess, 112.1ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 384)
Inference done.
Frame written to file.
0: 640x384 (no detections), 112.7ms
Speed: 2.1ms preprocess, 112.7ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
Inference done.
Frame written to file.
0: 640x384 (no detections), 118.9ms
Speed: 2.5ms preprocess, 118.9ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)
Inference done.
Frame written to file.
0: 640x384 (no detections), 114.1ms
Speed: 2.1ms preprocess, 114.1ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 384)
Inference done.
Frame written to file.
0: 640x384 (no detections), 113.0ms
Speed: 1.8ms preprocess